# Compute, in zonal means, how much of the ocean (per basin) has emerged at each year between 1861 to 2100

In [1]:
import os
import glob
from netCDF4 import Dataset as open_ncfile
import matplotlib.pyplot as plt
from maps_matplot_lib import defVarmme
from modelsDef import defModels
import numpy as np
import colormaps as cmaps
import datetime
import pickle

In [2]:
# ----- Workspace ------

indir_toe_rcphn = '/home/ysilvy/Density_bining/Yona_analysis/data/toe_zonal/toe_rcp85_histNat/hist_meanhistNat/'
indir_mme_rcp85 = '/data/ericglod/Density_binning/Prod_density_april15/mme_rcp85/'
indir_mme_hn = '/data/ericglod/Density_binning/Prod_density_april15/mme_histNat/'
indir_histrcp85 = '/home/ysilvy/Density_bining/Yona_analysis/data/hist_rcp85/'

# ----- Work -----
varname = defVarmme('salinity'); v = 'S'
iniyear = 1860
finalyear = 2100
models = defModels()

multstd=2

# ----- Variables ------
var = varname['var_zonal_w/bowl']
legVar = varname['legVar']
unit = varname['unit']

# Read latitude and density from original file
fileh_2d = '/data/ericglod/Density_binning/Prod_density_april15/mme_hist/' \
       'cmip5.multimodel_Nat.historical.ensm.an.ocn.Omon.density_zon2D.nc'
fh2d = open_ncfile(fileh_2d, 'r')
lat = fh2d.variables['latitude'][:]; latN = len(lat)
density = fh2d.variables['lev'][:]; levN = len(density)
basinN = 4


In [4]:
# --------------------------------------------------------------------------
# ----- Read ToE for each model and determine percentage of emergence ------
# --------------------------------------------------------------------------

# == Historical + RCP8.5 vs. historicalNat or vs. PiControl ==

nruns = 0 # Initialize total number of runs
nrunmax = 100

# -- Initialize varToE containing ToE of all runs
varToEA = np.ma.masked_all((nrunmax, levN, latN))
varToEP = np.ma.masked_all((nrunmax, levN, latN))
varToEI = np.ma.masked_all((nrunmax, levN, latN))

# Loop over models
listfiles = sorted(glob.glob(indir_toe_rcphn + '*'+legVar+'_toe_zonal*.nc'))
nmodels = len(listfiles)

nMembers = np.ma.empty(nmodels) # Initialize array for keeping number of members per model

# -- Initialize emerge/emerge_med containing percentage of emergence of all runs / of all models (median of members)
emerge = np.ma.masked_all((240,nrunmax, basinN))
emerge_med = np.ma.masked_all((240,nmodels, basinN)) # Median of members

for i in range(nmodels):
    file_toe = listfiles[i]
    ftoe = open_ncfile(file_toe, 'r')
    name = os.path.basename(file_toe).split('.')[1]

    # Fetch start and end indices for that model
    m=0
    while models[m]['name'] != name:
        m = m+1
    model = models[m]
    tstart = model['props'][2]
    tend = model['props'][3]

    # Read ToE (members, basin, density, latitude)
    if multstd == 1:
        toeread = ftoe.variables[var + 'ToE1'][:] + iniyear
    else:
        toeread = ftoe.variables[var + 'ToE2'][:] + iniyear
    nMembers[i] = toeread.shape[0]
    print('- Reading ToE of %s with %d members'%(name,nMembers[i]))
    nruns1 = int(nruns + nMembers[i])

    # Read signal
    signalread = ftoe.variables[var + '_change'][:]

    toeread.mask = signalread.mask

    # Save ToE
    varToEA[nruns:nruns1,:,:] = toeread[:,1,:,:]
    varToEP[nruns:nruns1,:,:] = toeread[:,2,:,:]
    varToEI[nruns:nruns1,:,:] = toeread[:,3,:,:]

    run_numbers = ftoe.variables['run_label'][:]
    
    for j in range(int(nMembers[i])):
        run_name = run_numbers[j]
        file2 = glob.glob(indir_histrcp85 + 'cmip5.' + name + '.' +'*'+run_name + '*zon2D.nc')[0]
        print('  '+run_name)
        
        # Read thickness of isopycnals in zonal means
        frun2 = open_ncfile(file2,'r')
        thick = np.ma.average(frun2.variables['isonthick'][tend:tend+95,:,:,:],axis=0) # In meters

        # Read bowl of corresponding run
        file_run = glob.glob(indir_histrcp85 + 'cmip5.' + name + '.'+'*'+run_name + '*zon1D.nc')[0]
        frun = open_ncfile(file_run,'r')
        bowl = np.ma.average(frun.variables['ptopsigma'][tend+95-20:tend+95,:,:],axis=0) # Average 2080-2100
        
        # Initialize array indicating each grid cell that has emerged for each year (1 or 0)
        flag = np.ma.masked_all((240,basinN,levN,latN))
        
        # Temporal loop
        for t in range(240):
            flag[t,:,:,:] = np.ma.where(toeread[j,:,:,:]<=t+1+iniyear,1,0) # 1 where ToE<=t1, 0 elsewhere

            # Mask points above bowl, in order to take into account only the "thickness" of ocean under the bowl in the percentage of emergence
            for ilat in range(len(lat)):
                if np.ma.is_masked(bowl[1,ilat]) == False :
                    inda = np.ma.nonzero(bowl[1,ilat]>=density)
                    flag[t,1,inda,ilat] = np.ma.masked
                    thick[1,inda,ilat] = np.ma.masked
                if np.ma.is_masked(bowl[2,ilat]) == False :
                    indp = np.ma.nonzero(bowl[2,ilat]>=density)
                    flag[t,2,indp,ilat] = np.ma.masked 
                    thick[2,indp,ilat] = np.ma.masked
                if np.ma.is_masked(bowl[3,ilat]) == False :
                    indi = np.ma.nonzero(bowl[3,ilat]>=density)
                    flag[t,3,indi,ilat] = np.ma.masked
                    thick[3,indi,ilat] = np.ma.masked

            # Cumulative thickness of emerged cells
            thick_t = np.ma.sum(flag[t,:,:,:]*thick,axis=(1,2))
            # Cumulative thickness of the whole basin
            thick_total = np.ma.sum(thick,axis=(1,2))

            # Save percentage of emergence 
            emerge[t,nruns+j,:] = 100*thick_t/thick_total
            
    emerge_med[:,i,:] = np.ma.median(emerge[:,nruns:nruns+1,:],axis=1)
    nruns = nruns1
        
print('Total number of runs:', nruns)
varToEA = varToEA[0:nruns,:,:]
varToEP = varToEP[0:nruns,:,:]
varToEI = varToEI[0:nruns,:,:]
emerge = emerge[:,0:nruns,:]

nruns = int(nruns)

- Reading ToE of CCSM4 with 6 members
  r1i1p1
  r2i1p1
  r3i1p1
  r4i1p1
  r5i1p1
  r6i1p1
- Reading ToE of CESM1-CAM5 with 3 members
  r1i1p1
  r2i1p1
  r3i1p1
- Reading ToE of CSIRO-Mk3-6-0 with 9 members
  r1i1p1
  r2i1p1
  r3i1p1
  r4i1p1
  r5i1p1
  r6i1p1
  r7i1p1
  r8i1p1
  r9i1p1
- Reading ToE of CanESM2 with 5 members
  r1i1p1
  r2i1p1
  r3i1p1
  r4i1p1
  r5i1p1
- Reading ToE of FGOALS-g2 with 1 members
  r1i1p1
- Reading ToE of GISS-E2-R with 3 members
  r1i1p1
  r2i1p1
  r2i1p3
- Reading ToE of HadGEM2-ES with 2 members
  r1i1p1
  r3i1p1
- Reading ToE of IPSL-CM5A-LR with 3 members
  r2i1p1
  r3i1p1
  r4i1p1
- Reading ToE of IPSL-CM5A-MR with 1 members
  r1i1p1
- Reading ToE of MIROC-ESM-CHEM with 1 members
  r1i1p1
- Reading ToE of MIROC-ESM with 1 members
  r1i1p1
Total number of runs: 35


In [5]:
# Save result
pickle.dump( emerge_med, open( "/home/ysilvy/Density_bining/Yona_analysis/data/percentage_emergence_medians_meanhistNat.pkl", "wb" ) )